# 트위터 크롤링

In [150]:
from twitter_api import BEARER_TOKEN

import tweepy
import pandas as pd

In [151]:
client = tweepy.Client(BEARER_TOKEN)

In [54]:
# 크롤링 하기
query = "#endangered -is:retweet lang:en"
tweets_en = tweepy.Paginator(
    client.search_recent_tweets, query=query, tweet_fields=["context_annotations", "created_at"], max_results=100
).flatten(limit=1000)

# 파일에 쓰기
f = open("tweet_en.txt", 'w', encoding="utf-8")
for tweet in tweets_en:
    f.write(tweet.text)
f.close()

In [154]:
query = "#생물다양성 -is:retweet lang:ko"
tweets_ko = tweepy.Paginator(
    client.search_recent_tweets, query=query, tweet_fields=["context_annotations", "created_at"], max_results=100
).flatten(limit=1000)

# 파일에 쓰기
f = open("tweet_ko.txt", 'w', encoding="utf-8")
for tweet in tweets_ko:
    f.write(tweet.text)
f.close()

# 형태소 분석

In [67]:
# 모듈 다운
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [125]:
# 파일 읽기
with open("tweet_en.txt", "r", encoding="utf-8") as f:
    data = f.read()

In [130]:
# 토큰화
data = data.lower()
token = nltk.word_tokenize(data)

# 불용어 처리
import string
stopwords = nltk.corpus.stopwords.words('english') + list(string.punctuation)
token = [t for t in token if t.lower() not in stopwords]

# 형태소 태깅
tag = nltk.tag.pos_tag(token)
word_data = []
for word, tag in tag:
    if tag in ["NN", "NNS", "NNPS", "VB", "VBD", "VBG", "VBN", 'VBZ', "JJ", 'JJR', 'JJS']:
        word_data.append(word)

print(word_data)

['missing', 'endangered', 'kathleen', 'michelsen', 'shaw', 'left', 'miller', 'ale', 'house', 'sanford', 'amp', 'suffers', 'white', 'female', 'grey', 'hair', 'wears', 'glasses', 'last', 'seen', 'wearing', 'blue', 'cardigan', 'dark', 'blue', 'patterned', 'skirt', 'amp', 'black', 'shoes', 'https', '//t.co/np7jf9fvk1the', 'documentary', 'endangered', 'ronanfarrow', 'hbo', 'see', 'great', 'work', 'day', 'age', 'spiresecology', 'hi', 'dr', 'gelling', 'thought', 'share', 'single', 'condensed', 'video', 'observations', 'endangered', 'water', 'vole', 'found', 'frays', 'farm', 'meadow', 'colne', 'valley', 'videos', 'twitter', 'page', 'evidence', 'sssi', 'alluvial', 'grassland', 'thriving', 'https', '//t.co/ileae7ptte', 'stae_elephants', 'meganmccubbin', 'salmahayek', 'karma', 'please', 'allow', 'elephant', 'freedom', 'deserves', 'needs', 'endcaptivity', 'amp', 'protect', 'elephants-you', 'disband', 'endangered', 'protected', 'respected', 'angie_karan', 'fuck', 'family', 'baby', 'elephants', 'end

In [144]:
# 숫자 세기
from collections import Counter
word_data = Counter(word_data)
make_word_cloud = word_data.most_common(70)

In [148]:
make_csv = pd.DataFrame(make_word_cloud)
make_csv.to_csv("tweet_en_word.txt")